# String utils

## word matching

In [ ]:
import string
import random
import re 

from typing import Sequence, Iterable, List
import numpy as np

In [ ]:
sentences = ['the Big cat went to the bar', 
             'the cat liked the big piano', 
             'the big fish never eats the bigger fish']

In [ ]:
reg = re.compile(r'(?i)\bbig\b')


In [ ]:
reg.findall(sentences[0])

In [12]:
import numpy as np 
%timeit x = np.ones(5_000_000, dtype=bool)

128 µs ± 5.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
%timeit x * x

171 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
import pyarrow as pa
x_pa = pa.array(x)

In [11]:
%timeit pa.compute.and_(x_pa,x_pa)

37.9 µs ± 467 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
get_indices_sentences_matching_words_regex(words=['big', 'cat'], sentences=sentences)

Now we can run the parallel version

In [ ]:
%%timeit
get_indices_sentences_matching_words_regex_parallel(words=['big', 'cat'], sentences=sentences, n_jobs=3)

Now let us try a big example

In [ ]:
pool_characters = string.ascii_lowercase

def create_random_word(min_chars=2, max_chars=15, pool_characters=None):
    n_chars = random.randint(min_chars,max_chars)
    return ''.join((random.choice(pool_characters) for x in range(n_chars)))

def create_random_sentence(n_words,pool_characters):
    return ' '.join( [create_random_word(pool_characters=pool_characters) for word in range(n_words) ])


In [ ]:
create_random_word(pool_characters= pool_characters)

In [ ]:
create_random_sentence(10,pool_characters)

In [ ]:
words = 'hi' 

In [ ]:
%%time
n_sentences = 2_000_000
n_words = 7
corpus = [create_random_sentence(7, pool_characters) for x in range(n_sentences)]

In [ ]:
corpus[0:10]

In [ ]:
corpus_14million = 7*[*corpus]

In [ ]:
%%time
get_indices_sentences_matching_words_regex(words=['big'], sentences=corpus_14million);

In [ ]:
res = get_indices_sentences_matching_words_regex(words=['big'], sentences=corpus_14million);

In [ ]:
res_par = get_indices_sentences_matching_words_regex_parallel(words=['big'], sentences=corpus_14million);

In [ ]:
%%time
get_indices_sentences_matching_words_regex_parallel(words=['big'], sentences=corpus_14million);

In [ ]:
res == res_par

### testing pyarrow

In [ ]:
import pyarrow

In [ ]:
pyarrow.set_cpu_count(1)

In [ ]:
from pyarrow.compute import extract_regex

In [ ]:
import re
pattern = re.compile(r'\bbig\b', re.IGNORECASE)

In [ ]:
res = extract_regex(corpus_14million, pattern=r'\bbig\b')

In [ ]:
res.is_valid().sum()

In [ ]:
res.is_valid() * res.is_valid()

In [ ]:
pyarrow.compute.and(res.is_valid(),res.is_valid())

In [ ]:
import pyarrow as pa

x = pa.array([True]*1000)
y = pa.array([False]*1000)


In [ ]:
%timeit x = pa.array([True]*1000_000)

In [ ]:
x = pa.array([True]*1000_000)

In [ ]:
import numpy as np

In [ ]:
%timeit x = pa.array(np.ones(1_000_000, dtype=bool))

In [ ]:
np.ones(1_000_000, dtype='bool')

In [ ]:
%timeit x = pa.array(np.ones(1_000_000))

In [ ]:
x = pyarrow.compute.and_(x,y)

In [ ]:
x

In [ ]:
pyarrow.compute.and_(res.is_valid(),res.is_valid())

In [ ]:
pyarrow.compute.and_(res.is_valid(),res.is_valid())

In [ ]:
np.where(np.array(res.is_valid()))[0].tolist()

In [ ]:
%%time
res = extract_regex(corpus_14million, pattern=r'\bbig\b/i')

In [ ]:
%%time
res_arrow = extract_regex(corpus_14million, pattern=r'(?i)\bbig\b')
res_arrow_np = np.array(res_arrow.is_valid())
indices = np.where(res_arrow_np)[0]

In [ ]:
res_arrow_np.sum()

In [ ]:
res_arrow_np

In [ ]:
res

In [ ]:
S = ["Big said the man", 'big boy','bigger than I want']
res_arrow = extract_regex(S,
                          pattern=r'(?i)\bbig\b')

In [ ]:
res_arrow

In [ ]:
get_indices_sentences_matching_words_regex(words=['big'], sentences=S )

In [ ]:
#np.where(res_arrow_np)[0]

## Test loading + regex 

In [ ]:
corpus_14million

In [ ]:
%timeit pyarrow.compute.sum(res_arrow.is_valid())

In [ ]:
res_par = get_indices_sentences_matching_words_regex_parallel(words=['big'], sentences=corpus_14million);

In [ ]:
len(res_par)

In [ ]:
#corpus_14million

In [ ]:
res = extract_regex(corpus_14million, pattern=r'big')

In [ ]:
len(res)

In [ ]:
import numpy as np
res_arrow = np.array(res_arrow.is_valid())

In [ ]:
res_arrow == np.array(res_par)

In [ ]:
len(np.where(res_arrow)[0])

In [ ]:
len(res_par)

In [ ]:
res_arrow.shape

In [ ]:
!open .

Adding more data

In [ ]:
corpus_40million = 20*[*corpus]

In [ ]:
%%time
res_arrow = extract_regex(corpus_40million,
                          pattern=r'(?i)\bbig\b')

In [ ]:
%%time
get_indices_sentences_matching_words_regex(words=['big'], sentences=corpus_40million);

In [ ]:
%%time
get_indices_sentences_matching_words_regex_parallel(words=['big'],
                                                    sentences=corpus_40million,
                                                    n_jobs=3);

In [ ]:
%%time
get_indices_sentences_matching_words_regex_parallel(words=['big'],
                                                    sentences=corpus_40million,
                                                    n_jobs=6);

In [ ]:
%%time
get_indices_sentences_matching_words_regex_parallel(words=['big'],
                                                    sentences=corpus_40million,
                                                    n_jobs=10);

In [ ]:
from gaia_science.common_base.dataframe_tools.utils import get_batches

In [ ]:
%%time
n_jobs = 8
n_sentences = len(corpus_40million)
n_batch = int(n_sentences/n_jobs)

partial_start_positions = [0] + list(np.cumsum([len(x) for x in get_batches(range(n_sentences), n_batch)]))

### Edit distance

In [ ]:
from gaia_science.common_base.string_utils.string_distances.edit_distance import EditDistanceInt

In [ ]:
d = EditDistanceInt(1,1,1)
d.evaluate('hi','ho')

### BKtree

In [ ]:
from gaia_science.common_base.string_utils.data_structures.bktree import BKTree

In [ ]:
import random
import string

def build_vocab(n_words, n_min_per_word, n_max_per_word):
    letters = string.ascii_lowercase
    random.seed(10)
    all_words = []
    for w in range(n_words):
        n = random.randint(n_min_per_word, n_max_per_word)
        all_words.append(''.join(random.choice(letters) for i in range(n)))
                         
    return all_words

In [ ]:
vocabulary = build_vocab(500_000, 2, 18)

In [ ]:
query = 'help'
max_dist = 1

In [ ]:
t1 = %timeit -o  candidates = [w for w in vocabulary if d.evaluate(query,w) <=max_dist]

In [ ]:
candidates.sort()
candidates

In [ ]:
bktree = BKTree(d.evaluate)

In [ ]:
%%time
bktree.fit(vocabulary)

In [ ]:
t2 = %timeit -o candidates = bktree.query(query, max_dist=max_dist)

In [ ]:
candidates.sort()
candidates

In [ ]:
t1.average/t2.average

In [ ]:
import pandas as pd

In [ ]:
results = pd.DataFrame()

In [ ]:
partial = pd.DataFrame({'a':[1,2,3],'b':['a','c','d']})

In [ ]:
partial

In [ ]:
results = pd.concat((results,partial))

In [ ]:
results

In [ ]:
results = pd.concat((results,partial))

In [ ]:
results.reset_index(drop=True)

In [ ]:
def append_many_df(initial_df, n=10_000):
    df_res = pd.DataFrame()
    for i in range(n):
        df_res = pd.concat((df_res, initial_df))
    return df_res.reset_index(drop=True)

In [ ]:
initial_df = pd.DataFrame({'a':[1,2,3],'b':['a','c','d']})

In [ ]:
%%time
append_many_df(initial_df)

In [ ]:
def append_many_df_as_list(initial_df, n=10_000):
    df_res = pd.DataFrame()
    for i in range(n):
        df_res = pd.concat((df_res, initial_df))
    return df_res.reset_index(drop=True)

In [ ]:
%%
append_many_df(initial_df)